In [97]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [98]:
df = pd.read_parquet("../../../data/dataframes/test_data.parquet")

In [99]:
df.head()

,gameCreation,gameEndTimestamp,gameDuration,gameVersion,kills,deaths,assists,championName,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,goldEarned,magicDamageDealtToChampions,physicalDamageDealtToChampions,teamEarlySurrendered,teamPosition,totalDamageDealtToChampions,totalTimeSpentDead,wardsKilled,wardsPlaced,win,teamId,laneMinionsFirst10Minutes,allyTopPick,allyJunglePick,allyMiddlePick,allyBottomPick,allySupportPick,enemyTopPick,enemyJunglePick,enemyMiddlePick,enemyBottomPick,enemySupportPick,laneOpponent,allyTopBan,allyJungleBan,allyMiddleBan,allyBottomBan,allySupportBan,enemyTopBan,enemyJungleBan,enemyMiddleBan,enemyBottomBan,enemySupportBan,Maxi,Moritz,Niclas,Aaron,totalAllyTeamKills,totalAllyTeamDeaths,gold@0,gold@1,gold@2,gold_diff@0,gold_diff@1,gold_diff@2,cs@0,cs@1,cs@2,deathBeforeMin5,remake,gold@3,gold@4,gold@5,gold@6,gold@7,gold@8,gold@9,gold@10,gold@11,gold@12,gold@13,gold@14,gold@15,gold@16,gold@17,gold@18,gold@19,gold@20,gold@21,gold@22,gold@23,gold@24,gold@25,gold@26,gold@27,gold@28,gold@29,gold@30,gold@31,gold@32,gold@33,gold@34,gold@35,gold@36,gold@37,gold@38,gold@39,gold@40,gold@41,gold_diff@3,gold_diff@4,gold_diff@5,gold_diff@6,gold_diff@7,gold_diff@8,gold_diff@9,gold_diff@10,gold_diff@11,gold_diff@12,gold_diff@13,gold_diff@14,gold_diff@15,gold_diff@16,gold_diff@17,gold_diff@18,gold_diff@19,gold_diff@20,gold_diff@21,gold_diff@22,gold_diff@23,gold_diff@24,gold_diff@25,gold_diff@26,gold_diff@27,gold_diff@28,gold_diff@29,gold_diff@30,gold_diff@31,gold_diff@32,gold_diff@33,gold_diff@34,gold_diff@35,gold_diff@36,gold_diff@37,gold_diff@38,gold_diff@39,gold_diff@40,gold_diff@41,cs@3,cs@4,cs@5,cs@6,cs@7,cs@8,cs@9,cs@10,cs@11,cs@12,cs@13,cs@14,cs@15,cs@16,cs@17,cs@18,cs@19,cs@20,cs@21,cs@22,cs@23,cs@24,cs@25,cs@26,cs@27,cs@28,cs@29,cs@30,cs@31,cs@32,cs@33,cs@34,cs@35,cs@36,cs@37,cs@38,cs@39,cs@40,cs@41,totalTeamGoldDiff@5,totalTeamGoldDiff@10,totalTeamGoldDiff@15,totalTeamGoldDiff@20,firstSuccessfulJunglerGank_s,totalKillsAtMinute5,totalKillsAtMinute10,totalKillsAtMinute15,totalKillsAtMinute20,gold@42,gold@43,gold_diff@42,gold_diff@43,cs@42,cs@43,gold@44,gold@45,gold@46,gold_diff@44,gold_diff@45,gold_diff@46,cs@44,cs@45,cs@46,gameDuration_m_s,onBlueSide,timeBetweenGames,session
matchId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
EUW1_6699278609,2023-11-30 23:50:46,2023-11-30 23:56:32,104,13.23,0,0,0,Brand,False,False,False,False,True,False,500,0,0,False,JUNGLE,0,0,0,1,True,100,0,Jax,Brand,Akshan,Draven,Pyke,Camille,LeeSin,Akali,Seraphine,Leblanc,LeeSin,Teemo,Braum,Belveth,Zed,JarvanIV,Yasuo,Belveth,Soraka,NoBan,Kalista,False,False,False,False,0,0,500,500,500,0,0,0,0,0,0,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01:44,True,NaT,0
EUW1_6640876214,2023-10-20 23:01:31,2023-10-20 23:42:28,2440,13.20,9,8,8,Gragas,False,False,False,False,False,False,16382,24173,1070,False,JUNGLE,26559,250,4,11,True,100,1,KSante,Gragas,Xerath,Caitlyn,Rakan,Camille,Briar,Lux,Ziggs,Pyke,Briar,Fiora,Graves,Hecarim,Orianna,Draven,Anivia,Belveth,Draven,Ivern,Jax,False,False,False,False,47,38,500,500,611,0,0,0,0,0,4,False,None,1022.0,1395.0,1638.0,1964.0,2271.0,2494.0,2905.0,3063.0,3290.0,3891.0,4609.0,5028.0,5315.0,5887.0,6486.0,7159.0,7559.0,7978.0,8415.0,8670.0,9316.0,9938.0,11179.0,11346.0,11473.

In [102]:
df["remake"]

matchId
EUW1_6699278609    True
EUW1_6640876214    None
EUW1_6640824488    None
EUW1_6640702123    None
EUW1_6640622281    None
EUW1_6640546278    None
EUW1_6640484348    None
EUW1_6640415940    None
EUW1_6640332312    None
EUW1_6640284724    None
EUW1_6636726874    None
EUW1_6635421105    None
EUW1_6635393755    None
EUW1_6635338378    None
EUW1_6634014214    None
EUW1_6633391422    None
EUW1_6633298048    None
EUW1_6633211520    None
EUW1_6633123006    None
EUW1_6630878909    None
EUW1_6630809772    None
EUW1_6630718713    None
EUW1_6630627409    None
EUW1_6629484737    None
EUW1_6628165340    None
EUW1_6628128122    None
EUW1_6628082919    None
EUW1_6628006057    None
EUW1_6626919610    None
EUW1_6626886699    None
EUW1_6626846472    None
EUW1_6626789250    None
EUW1_6626737291    None
EUW1_6626668338    None
EUW1_6626599960    None
EUW1_6626520348    None
EUW1_6625697356    None
EUW1_6625654095    None
EUW1_6625579965    None
EUW1_6625263523    None
EUW1_6625201503    None
EUW1_662

In [ ]:
my_list = [1, 2, 3]

def change(list):
    list.append(4)

change(my_list)